Setup

In [ ]:
import sys

sys.path.append("../src")

In [ ]:
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from scipy import interpolate
from sklearn import metrics

from src.plots import PlotData, plot_interpolation
from src import kernels
from src.conv import conv1d_interpolate

In [ ]:
sns.set()

Functions to interpolate

In [ ]:
def simple(x):
    return np.sin(x)

def inverted_sin(x):
    return np.sin(np.power(x, -1))

def sign(x):
    return np.sign(np.sin(8*x))

In [ ]:
x = np.linspace(1e-6, 2*np.pi, 10_000)  # start from small epsilon to avoid division by zero

y_simple = simple(x)
y_sin_func = inverted_sin(x)
y_sgn_func = sign(x)

_ = plt.figure(figsize=[12, 8])

_ = plt.plot(x, y_simple)
_ = plt.plot(x, y_sin_func)
_ = plt.plot(x, y_sgn_func)

Kernels

In [ ]:
x = np.linspace(-2, 2, 1000)

_ = plt.figure(figsize=[12, 8])

_ = plt.plot(x, kernels.sample_hold_kernel(x, offset=0, width=1))
_ = plt.plot(x, kernels.nearest_neighbour_kernel(x, offset=0, width=1))
_ = plt.plot(x, kernels.linear_kernel(x, offset=0, width=1))
_ = plt.plot(x, kernels.keys_kernel(x, offset=0, width=1))
_ = plt.plot(x, kernels.sinc_kernel(x, offset=0, width=1, alpha=np.inf))

_ = plt.legend(["Sample Hold", "Nearest", "Linear", "Keys", "Sinc"])

In [ ]:
n_samples = 10  # define each time to easily change and investigate results
n_predictions = 10_000

x = np.linspace(1e-6, 2*np.pi, n_samples)
y = simple(x)

x_interp = np.linspace(1e-6, 2*np.pi, n_predictions)
y_interp = conv1d_interpolate(x_measure=x, y_measure=y, x_interpolate=x_interp, kernel=kernels.sinc_kernel)
y_true = simple(x_interp)

print(f"MSE: {metrics.mean_squared_error(y_pred=y_interp, y_true=y_true):.8f}")
plot_interpolation(measure=PlotData(x=x, y=y), interpolation=PlotData(x=x_interp, y=y_interp), real=PlotData(x=x_interp, y=y_true))

In [ ]:
n_samples = 10  # define each time to easily change and investigate results
n_predictions = 10_000

x = np.linspace(1e-6, 2*np.pi, n_samples)
y = inverted_sin(x)

x_interp = np.linspace(1e-6, 2*np.pi, n_predictions)
y_interp = conv1d_interpolate(x_measure=x, y_measure=y, x_interpolate=x_interp, kernel=kernels.keys_kernel)
y_true = inverted_sin(x_interp)

print(f"MSE: {metrics.mean_squared_error(y_pred=y_interp, y_true=y_true):.8f}")
plot_interpolation(measure=PlotData(x=x, y=y), interpolation=PlotData(x=x_interp, y=y_interp), real=PlotData(x=x_interp, y=y_true))

In [ ]:
n_samples = 10  # define each time to easily change and investigate results
n_predictions = 10_000

x = np.linspace(1e-6, 2*np.pi, n_samples)
y = sign(x)

x_interp = np.linspace(1e-6, 2*np.pi, n_predictions)
y_interp = conv1d_interpolate(x_measure=x, y_measure=y, x_interpolate=x_interp, kernel=kernels.linear_kernel)
y_true = sign(x_interp)

print(f"MSE: {metrics.mean_squared_error(y_pred=y_interp, y_true=y_true):.8f}")
plot_interpolation(measure=PlotData(x=x, y=y), interpolation=PlotData(x=x_interp, y=y_interp), real=PlotData(x=x_interp, y=y_true))

# Image Interpolation

Setup

In [ ]:
import sys

sys.path.append("../src")

In [ ]:
import fractions

import numpy as np
from numpy.typing import NDArray
from skimage import io, transform, color

from src.downsampling import rgb_downsample, nonlinear_downsample
from src.conv import image_interpolate, rgb_image_interpolate
from src import kernels

Data

In [ ]:
color_image = io.imread(r"../../data/image.jpg")
color_image = transform.resize(color_image, output_shape=(1000, 1000, 3))

grey_image = color.rgb2gray(color_image)  # convert to grey-scale to work with both
grey_image.shape, color_image.shape

((1000, 1000), (1000, 1000, 3))

Downsampling

In [ ]:
downsampled = rgb_downsample(color_image, kernel_size=2)
_ = io.imshow(downsampled)

In [ ]:
downsampled = nonlinear_downsample(grey_image, kernel_size=2, aggregate=np.max)
_ = io.imshow(downsampled)

Upsampling

In [ ]:
grey_image.shape

(1000, 1000)

In [ ]:
%%time

upsampled = image_interpolate(grey_image, kernel=kernels.linear_kernel, ratio=2)
upsampled.shape

(2000, 2000)

In [ ]:
_ = io.imshow(upsampled)

RGB image can be interpolated the same way by applying ineterpolation to each channel

In [ ]:
%%time

color_image = transform.resize(color_image, output_shape=(400, 400, 3))  # scale down to avoid long wait time
color_upsampled_image = rgb_image_interpolate(color_image, kernel=kernels.linear_kernel, ratio=2)
color_upsampled_image.shape

(800, 800, 3)

In [ ]:
_ = io.imshow(color_upsampled_image)